In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import precision_recall_fscore_support,accuracy_score

In [4]:
colnames = ['text','event_location','average_tone','article_date','article_ID','article_URL_1','MBFC_factuality_label_1','article_URL','MBFC_factuality_label','URL_to_MBFC_page','source_name','MBFC_notes_about_source','MBFC_bias_label','source_URL','propaganda_label']
train_df = pd.read_csv("data_proppy/proppy_1.0.train.tsv",sep='\t',names=colnames)[["text",'average_tone','source_name','propaganda_label']]
eval_df = pd.read_csv("data_proppy/proppy_1.0.dev.tsv",sep='\t',names=colnames)[["text",'average_tone','source_name','propaganda_label']]

train_df['propaganda_label'] = train_df['propaganda_label'].apply(lambda x: int(x==1))
eval_df['propaganda_label'] = eval_df['propaganda_label'].apply(lambda x: int(x==1))

pd.set_option('display.max_colwidth', 150)

train_df 

,text,average_tone,source_name,propaganda_label
0,"Et tu, Rhody? A recent editorial in the Providence Journal cataloged everything it could find wrong with Connecticut and ended with this suggesti...",-3.181818,The Hartford Courant,0
1,A recent post in The Farmington Mirror — our town’s version of The Onion — encouraged parents to take advantage of a shuttle service offered by th...,-0.424328,The Hartford Courant,0
2,"President Donald Trump, as he often does while responding to natural disasters, mass shootings or unfolding crises, spent much of his time congrat...",-2.469136,The Hartford Courant,0
3,"February is Black History Month, and nothing looms larger in black history than the evil specter of slavery. Three exhibits in the state take on t...",-0.894632,The Hartford Courant,0
4,"The snow was so heavy, whipped up by gusting winds, that travel was nearly impossible. Thousands abandoned their cars. Ambulances could not pass t...",-4.800000,The Hartford Courant,0
...,...,...,...,...
35981,"From The Telegraph: Towns in Brazil have become refugee camps for a tide of desperate Venezuelans 30 AUGUST 2018 • 6:00AM Johan Rodriguez, a buil...",-3.193277,lewrockwell.com,1
35982,The second episode of Consortium News on Flash Points focuses on two different perspectives on John McCain and the real meaning of Russian interfe...,0.526316,lewrockwell.com,1
35983,"It is beginning. Actually, it’s been happening for a long time – like a slowly metastasizing cancer. The afflicted can no longer hide the underly...",-3.455285,lewrockwell.com,1
35984,"Justin’s note: As regular Dispatch readers know, every Friday we feature no-filtered insights from Doug Casey. You see, Doug isn’t just a world-cl...",-1.052049,lewrockwell.com,1


In [5]:
counts = train_df[["source_name","propaganda_label","text"]].groupby("propaganda_label").count().rename(columns={"source_name":"Count","text":"frac"})
counts.frac = counts.frac/(counts.Count.sum())
counts

,Count,frac
propaganda_label,,
0,31965,0.888262
1,4021,0.111738


In [6]:
# get nltk stopwords 
stops = set(stopwords.words('english'))

# verify that we can pipeline these together. 
[word for word in word_tokenize("All work and no play makes jack a dull boy.") if word not in stops]

['All', 'work', 'play', 'makes', 'jack', 'dull', 'boy', '.']

In [7]:
# now vectorize our traain set... 
vectorizer = CountVectorizer(lowercase=True,
							tokenizer=word_tokenize,
							stop_words=stops,
							ngram_range=(1,3),    	# might want to hyperparameter tune this
							max_df=0.9,				# might want to hyperparameter tune this 
							min_df=0.001,				# might want to hyperparameter tune this
							)
X_train = vectorizer.fit_transform(train_df["text"].to_list())
X_eval = vectorizer.transform(eval_df["text"].to_list())
X_train

/Users/zachary/opt/anaconda3/envs/eecs_nlp/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


<35986x60239 sparse matrix of type '<class 'numpy.int64'>'
	with 12610702 stored elements in Compressed Sparse Row format>

In [8]:
# Now create our fun lil naieve bayes model
classifier = GaussianNB()
classifier.fit(X_train.toarray(),train_df['propaganda_label'].to_numpy())

classifier

GaussianNB()

In [8]:
# for uni-bi-gram models
preds = classifier.predict( X_eval.toarray()) 
eval_df["preds"] = preds 
prec,recall,f_1,sup = precision_recall_fscore_support(eval_df.propaganda_label,eval_df.preds)
accuracy = accuracy_score(eval_df.propaganda_label,eval_df.preds)
stats_arr = {"Precision": prec[1], "recall" : recall[1], "f1" : f_1[1],"Accuracy": accuracy}
stats = pd.DataFrame(stats_arr, ['score'])
stats



,Precision,recall,f1,Accuracy
score,0.401361,0.513043,0.450382,0.859512


In [10]:
# for uni-bi-tri-gram models
preds = classifier.predict( X_eval.toarray()) 
eval_df["preds"] = preds 
prec,recall,f_1,sup = precision_recall_fscore_support(eval_df.propaganda_label,eval_df.preds,average='macro')
accuracy = accuracy_score(eval_df.propaganda_label,eval_df.preds)
stats_arr = {"Precision": prec[1], "recall" : recall[1], "f1" : f_1[1],"Accuracy": accuracy}
stats = pd.DataFrame(stats_arr, ['score'])
stats




IndexError: invalid index to scalar variable.

In [ ]:
# now also train a basic classifier on those same features. 
